In [1]:
from tqdm import tqdm, trange
import json
from collections import namedtuple, defaultdict
import pandas as pd
import numpy as np
import torch
import os
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models.word2vec import Word2Vec
import random
from scipy.sparse import csr_matrix
import dgl

import pickle as pkl
import sys
sys.path.append('/mnt/nfs/zhangtl/utils')
from util import myout

Using backend: pytorch


## load

In [2]:
dataset = 'env'
df = pd.read_csv(f'../../../01_process/data/{dataset}/Papers.csv') # 5353904
df.dropna(subset=['p_id', 'year'], inplace=True)
df.fillna({'refs': '', 'a_ids': '', 'v_id': '', 'fids': '', 'abst': ''}, inplace=True)
df

/home/ztl/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
0,2128057,Eel culture in Europe: past and present state,3.007549e+09,1800.0,0,,2602168878,201530359.0,"32,34,60",
1,1878511,"Review of Marco d’Eramo, ""The Pig and the Skys...",2.894669e+09,1800.0,0,,2895435177,50366009.0,"305,96,1010",
2,1126716,"Anniversary Meeting, January 25th, 1836",2.131502e+09,1836.0,0,,2677589731,30982003.0,"20,2,32",
3,1214221,"Anniversary Meeting, January 26th, 1835",2.160986e+09,1836.0,0,,2708792614,30982003.0,"20,2,32",
4,1158010,"April 6th, 1835",2.141970e+09,1836.0,0,,2658861684,30982003.0,"32,2",
...,...,...,...,...,...,...,...,...,...,...
2213974,2208716,The Role of Energy Poverty on Economic Growth ...,3.038055e+09,2021.0,0,,"3037191078,3038127954",2764595475.0,"21298,28255,140,2281",
2213975,2211822,The use of artificial neural networks to predi...,3.039621e+09,2021.0,0,,"2713294834,2321326026,3040007285,2563563244",84339314.0,"466,33511,218,805,69,34,220",
2213976,2213134,Trichoderma viride involvement in the sorption...,3.040292e+09,2021.0,0,"157653275,1606745945,1900140751,1964397481,197...","2945760575,2945760575,2899355254,2244066162,29...",145089992.0,"9064,35,14183,17254,16378,34,214,27373,175,39",
2213977,2207140,Using electromagnetic fields to inhibit biofou...,3.037531e+09,2021.0,0,"768576097,1775749144,1968834637,1978576748,198...","2890473624,3008781678,3004236009,2901372249,26...",145089992.0,"35,458,5997,34,6613,247,906,5343,5234,1284",


In [3]:
start_year, end_year = 2000, 2022
drop_id = df[(df.year < start_year) | (df.year >= end_year)].index
print(f'Drop {len(drop_id)} rows') # 559749
papers = df.drop(drop_id) # 3547589
papers

Drop 576824 rows


,Unnamed: 0,title,p_id,year,ncites,refs,a_ids,v_id,fids,abst
576825,1338336,"""...sich diesen Raum zu nehmen und sich freizu...",2.315067e+09,2000.0,3,,"2528884061,2527858113",,,
576826,1170610,"""A Place That's Good,"" Gitksan Landscape Perce...",2.146234e+09,2000.0,62,"562777957,588240888,599706738,1273791837,15399...",2110575733,82712235.0,"96,32,1556,1309,656,341,5525,1552,21175,190",
576827,1860432,"""Airs, Waters, Places"" and Other Hippocratic W...",2.887406e+09,2000.0,9,,"2636099610,2886014895",33960400.0,"8836,17,34,16547",
576828,1101157,"""BACWAVE,"" a spatial-temporal model for travel...",2.122931e+09,2000.0,53,"36425615,1518887107,1548641366,1554346626,1592...","2089102391,2134779190,2805394381",97758720.0,"32,499,1468,426,1090,516,61,60,2,11570,4378",
576829,1402041,"""Collision of Evidence and Assumptions: TMI De...",2.331378e+09,2000.0,1,,"2036082374,2655364972,2045858341,318118501,191...",10446503.0,"2,61438,17,307,14357",
...,...,...,...,...,...,...,...,...,...,...
2213974,2208716,The Role of Energy Poverty on Economic Growth ...,3.038055e+09,2021.0,0,,"3037191078,3038127954",2764595475.0,"21298,28255,140,2281",
2213975,2211822,The use of artificial neural networks to predi...,3.039621e+09,2021.0,0,,"2713294834,2321326026,3040007285,2563563244",84339314.0,"466,33511,218,805,69,34,220",
2213976,2213134,Trichoderma viride involvement in the sorption...,3.040292e+09,2021.0,0,"157653275,1606745945,1900140751,1964397481,197...","2945760575,2945760575,2899355254,2244066162,29...",145089992.0,"9064,35,14183,17254,16378,34,214,27373,175,39",
2213977,2207140,Using electromagnetic fields to inhibit biofou...,3.037531e+09,2021.0,0,"768576097,1775749144,1968834637,1978576748,198...","2890473624,3008781678,3004236009,2901372249,26...",145089992.0,"35,458,5997,34,6613,247,906,5343,5234,1284",


## build graph

In [4]:
dataset = 'env'
p_id2emb = pkl.load(open(f'../../../01_process/data_papers/{dataset}/p_id2emb.pkl', 'rb'))
myout(p_id2emb)
def update_idx(idx, dic, cnt, feats, rel, feat_dim, no_emb):
    if idx not in dic:
        dic[idx] = cnt
        cnt += 1
        if rel == 0:
            try:
                feats.append(torch.from_numpy(p_id2emb[idx]).to(torch.float32))
            except:
                feats.append(torch.rand(feat_dim).to(torch.float32)-1)
                no_emb += 1
        elif rel == 1:
            feats.append(torch.rand(feat_dim).to(torch.float32)+1)
        elif rel == 2:
            feats.append(torch.rand(feat_dim).to(torch.float32)+2)
        elif rel == 3:
            feats.append(torch.rand(feat_dim).to(torch.float32))
    return dic, cnt, feats, no_emb

p_id2emb : len=2213957, dict([3007548847: [-0.05492322 -0.22065584  0.2985575  -0.15915087 -0.02410255 -0.2155301
 -0.20967305  0.37147653 -0.04364195 -0.07630312  0.2680815   0.11518687
  0.25646588 -0.00197192 -0.27339405  0.08477961  0.12236243  0.10521181
  0.13921672  0.3503011   0.1794407  -0.02331379  0.07343214 -0.02041487
 -0.06727359 -0.22557878 -0.18949904  0.11083816 -0.23937304  0.18753085
 -0.3167712   0.07566749  0.00274131 -0.00359801  0.00646017 -0.05402936
  0.19669741 -0.09096984  0.3734789  -0.00901081 -0.05785743  0.16725983
 -0.33260292  0.33185875  0.23899257  0.03505135  0.19632745 -0.11480837
  0.20247723  0.2863608   0.39671952  0.08246421  0.04187625 -0.0078773
  0.17659152  0.16527024  0.12494088  0.09159656 -0.10235982  0.5952348
  0.12883677  0.04587729 -0.0088445  -0.03463843 -0.24606697 -0.15229443
  0.19952697  0.39229855  0.6941978  -0.1300182   0.12201749  0.04031594
 -0.3944111  -0.5086687   0.03052625  0.05774899  0.2541784   0.08053086
  0.19449656

In [5]:
start_year, end_year = 2000, 2022
feat_dim = 128

id2nid, cnt, no_emb = {}, 0, 0
lst, feats = [], []
for i in trange(len(papers)):
    ts = int(papers['year'].iloc[i])
    p_id = int(papers['p_id'].iloc[i])
    refs = papers['refs'].iloc[i]
    
    id2nid, cnt, feats, no_emb = update_idx(p_id, id2nid, cnt, feats, 0, feat_dim, no_emb)
    if len(refs)>0:
        rlst = refs.split(',')
        for ref in rlst:
            if ref != '':
                ref = int(ref)
                id2nid, cnt, feats, no_emb = update_idx(ref, id2nid, cnt, feats, 0, feat_dim, no_emb)
                lst.append((id2nid[p_id], id2nid[ref], 0, ts))
        
feat = torch.stack(feats)
src = torch.tensor([item[0] for item in lst])
tgt = torch.tensor([item[1] for item in lst])
rel = torch.tensor([item[2] for item in lst])
tsp = torch.tensor([item[3] for item in lst])
myout(feat, src, tgt, rel, tsp)

100%|██████████| 1637142/1637142 [04:40<00:00, 5841.82it/s]


feat : shape=torch.Size([10430502, 128])
tensor([[-0.2566, -0.0625, -0.0731,  ...,  0.0511, -0.4368, -0.1893],
        [-0.2766, -0.1009,  0.1672,  ...,  0.2571, -0.0277,  0.0747],
        [-0.3123, -0.5056, -0.1238,  ..., -0.2600, -0.7426, -0.4625],
        ...,
        [-0.5028, -0.4492, -0.5358,  ..., -0.3199, -0.0595, -0.7186],
        [-0.5698, -0.4317, -0.1866,  ..., -0.4223, -0.0234, -0.9022],
        [-0.2174, -0.2687,  0.1125,  ...,  0.0616,  0.0618,  0.2382]])
src : shape=torch.Size([46221685]), tensor([       1,        1,        1,  ..., 10430495, 10430495, 10430495])
tgt : shape=torch.Size([46221685]), tensor([       2,        3,        4,  ..., 10174006, 10430500,  9912505])
rel : shape=torch.Size([46221685]), tensor([0, 0, 0,  ..., 0, 0, 0])
tsp : shape=torch.Size([46221685]), tensor([2000, 2000, 2000,  ..., 2021, 2021, 2021])


In [6]:
graph = dgl.graph((src, tgt), num_nodes=len(feat))
graph.ndata['feat'] = feat

nid2id = {vv: kk for kk, vv in id2nid.items()}
graph.ndata['raw_nid'] = torch.arange(len(feat))

graph.edata['rel'] = rel
graph.edata['ts'] = tsp
graph

Graph(num_nodes=10430502, num_edges=46221685,
      ndata_schemes={'feat': Scheme(shape=(128,), dtype=torch.float32), 'raw_nid': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rel': Scheme(shape=(), dtype=torch.int64), 'ts': Scheme(shape=(), dtype=torch.int64)})

In [7]:
dgl.save_graphs(f'../data/{dataset}/graph.bin', [graph])

In [8]:
json.dump(id2nid, open(f'../data/{dataset}/id2nid.json', 'w'))

## gen cites

In [9]:
cites = {}
print(start_year, end_year)
for year in range(start_year, end_year):
    cites[year] = defaultdict(int)

papers.fillna({'refs': ''}, inplace=True)
for i in trange(len(papers)):
    year = int(papers['year'].iloc[i])
    refs = papers['refs'].iloc[i]
    if len(refs)>0:
        rlst = refs.split(',')
        rlst = [int(item) for item in rlst]
        for ref in rlst:
            if ref != '':
                cites[year][ref] += 1

2000 2022


100%|██████████| 1637142/1637142 [01:08<00:00, 23872.06it/s]


In [10]:
tsp = graph.edata['ts']
ts_vals, ts_cuts = np.unique(tsp.numpy(), return_index=True)
ts_cuts = list(ts_cuts) + [len(tsp.numpy())]

num_ts = len(ts_vals)
ts_infos = np.stack([ts_vals, ts_cuts[0:num_ts], ts_cuts[1:num_ts+1]]).transpose()
myout(ts_cuts, ts_vals, ts_infos)

ts_cuts : len=23, list([0, 523027, 1088215, ..., 41859728, 46219744, 46221685])
ts_vals : shape=(22,), [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
ts_infos : shape=(22, 3)
[[    2000        0   523027]
 [    2001   523027  1088215]
 [    2002  1088215  1725447]
 [    2003  1725447  2428878]
 [    2004  2428878  3229682]
 [    2005  3229682  4139968]
 [    2006  4139968  5233807]
 [    2007  5233807  6468567]
 [    2008  6468567  7889933]
 [    2009  7889933  9539843]
 [    2010  9539843 11346921]
 [    2011 11346921 13410165]
 [    2012 13410165 15659600]
 [    2013 15659600 18274925]
 [    2014 18274925 21125810]
 [    2015 21125810 24396063]
 [    2016 24396063 28074533]
 [    2017 28074533 32103482]
 [    2018 32103482 36503974]
 [    2019 36503974 41859728]
 [    2020 41859728 46219744]
 [    2021 46219744 46221685]]


In [11]:
labels = {}
nid2id = {v:k for k,v in id2nid.items()}
for year in range(start_year, end_year):
    left, right = ts_infos[np.where(ts_infos[:, 0]==year)[0][0], 1:]
    nids = graph.edges()[0][left:right].unique().tolist()
    ids = [nid2id[nid] for nid in nids]
    
    pdf = pd.DataFrame({'id': ids, 'nid': nids})
    tbar = trange(year+1, end_year, desc=str(year))
    for yy in tbar:
        cdf = pd.DataFrame({'id': list(cites[yy].keys()), str(yy): list(cites[yy].values())})
        cdf[str(yy)] = cdf[str(yy)].astype('float32')
        
        pdf = pd.merge(pdf, cdf, how='left', on='id')
        tbar.set_postfix(year=year, pdf=len(pdf))
    pdf.fillna(0, inplace=True)
    labels[year] = pdf
    

2020: 100%|██████████| 1/1 [00:00<00:00, 53.61it/s, pdf=1e+5, year=2020]
2021: 0it [00:00, ?it/s]


In [12]:
pkl.dump(labels, open(f'../data/{dataset}/labels.pkl', 'wb'))

## cum log labels

In [13]:
def cumulative_log(df):
    colsn = list(df.columns)
    for i in range(3, len(colsn)):
        df[colsn[i]] += df[colsn[i-1]]
    df.iloc[:, 2:] = np.log(df.iloc[:, 2:] + 1)
    return df

labels_cum_log = {}
for year in range(start_year, end_year-2): # 2000, 2020
    labels_cum_log[year] = cumulative_log(labels[year])
labels_cum_log[end_year-2] = labels[end_year-2]
print(len(labels_cum_log))
labels_cum_log[2005]

21


,id,nid,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2160554188,5451,0.000000,0.000000,0.000000,0.693147,1.386294,1.609438,1.945910,2.079442,2.197225,2.484907,2.639057,2.708050,2.833213,3.091043,3.218876,3.218876
1,2053733118,156268,0.000000,1.098612,1.098612,1.386294,1.609438,1.609438,1.791759,1.791759,1.791759,2.079442,2.079442,2.079442,2.197225,2.197225,2.397895,2.397895
2,1998017163,319456,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,1.609438,1.609438,1.609438,1.609438,1.945910,1.945910,1.945910,1.945910
3,2089177392,345246,0.000000,0.000000,0.693147,1.098612,1.609438,1.945910,2.484907,2.564949,2.708050,2.833213,2.944439,3.044523,3.044523,3.091043,3.091043,3.091043
4,2096556645,535291,1.098612,1.791759,2.079442,2.397895,2.564949,2.639057,2.639057,2.639057,2.708050,2.772589,2.944439,2.995732,3.178054,3.258096,3.295837,3.295837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32558,2049329352,1730861,0.000000,1.098612,2.197225,2.564949,2.708050,2.772589,2.890372,2.890372,2.890372,2.944439,2.995732,3.091043,3.091043,3.135494,3.135494,3.135494
32559,237814319,1730897,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
32560,582115453,1730925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.098612,1.098612,1.098612
32561,2071392671,1730937,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
pkl.dump(labels_cum_log, open(f'../data/{dataset}/labels_cum_log.pkl', 'wb'))